# Trabajo práctico 

Nombres de integrantes/padron:
##### Braian
##### Pablo
##### Prieto, Pablo
##### Ruiz, Ariel 

In [ ]:
# magic function para hacer que los graficos de matplotlib se renderizen en el notebook.
%matplotlib inline

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



##  Análisis de columnas

#### Se hace un pequeño análisis global del dataset.

In [ ]:
#Ante todo, empezamos a analizar la información brindada por los datasets y sus respectivas columnas.
sells = pd.read_csv('data/properati-AR-2017-01-01-properties-sell.csv')
sells.columns.values

In [ ]:
#Se analizan las columnas que consideramos necesarias para nuestro analisis exploratorio.
#Con esto borro las columnas del dataframe que no aporten algún valor significativo a nuestro entender (id, operation, place_with_parents_name, country_name, geonames_id(ver si sirve)
# lat-lon, price_aprox_local_currency, price_usd_per_m2, price_per_m2, floor, description, title, image_thumbnail)
sells = sells.drop(sells.columns[[0, 2, 5, 6, 8, 9, 14, 19, 20, 24, 25, 26]], axis=1)

Se van eliminando algunas columnas que a nuestro entender no agregan información relevante para el estudio.

In [ ]:
#Vericamos las columnas restantes a analizar.
sells.columns.values

In [ ]:
#Mirada global del dataframe resultante.
sells.head(10)

In [ ]:
#Se empieza por analizar la columna donde se encuentra el lugar de la publicación.
#uso regex para buscar coincidencias por string por Capital Federal y se hace un count para consistencias de datos. 
sells_caba = sells.loc[sells['state_name'].str.contains(r'(Capital Federal|capital federal)')]
sells_caba.count()

In [ ]:
#filtro barrio Capital Federal de caba (se analiza de forma aparte)
sells_caba = sells_caba[sells_caba['place_name']!='Capital Federal']
sells_caba.count()

In [ ]:
#Uso regex para buscar coincidencias por string entre diferentes zonas del Gran Buenos Aires y las agrupo.
#Se hace un count para consistencias de datos.
sells_gba = sells.loc[sells['state_name'].str.contains(r'(G.B.A|Zona Norte|Zona Sur|Zona Este|Zona Oeste)')]
sells_gba.count()

### Se observa el enfoque en particular que tiene el dataframe.

In [ ]:
#Distribución de ventas agrupadas por las zonas de Capital Federal y el Gran Buenos Aires.
ventas_en_caba = sells_caba['state_name'].value_counts()
ventas_en_gba = sells_gba['state_name'].value_counts().sum()
sizes = (ventas_en_caba,ventas_en_gba)
ventas_en_gba
nombres_de_parametros = ['Ventas en Capital Federal', 'Ventas en Gran Buenos Aires']

plt.figure(figsize=(8, 8))
plt.title('Distribucion de ventas de propiedades por zonas\n(Capital Federal y el gran Buenos Aires)', fontsize=20)
plt.pie(sizes, labels=nombres_de_parametros, autopct='%1.1f%%', startangle=1000
    , colors=['seagreen', 'coral'], explode=(0.15, 0))
plt.show()

Con este plot, se puede ver que el dataframe tiene un enfoque particular en ventas que se centra en zonas aledañas a Capital Federal.

### Primeros trabajos sobre propiedades en Capital Federal.

In [ ]:

#Obtengo del dataframe de Capital Federal las columnas 'state_name', 'place_name' y 'price_aprox_usd'.
#Muestra un random de 10 publicaciones.
sells_caba_price = sells_caba.loc[:,['state_name','place_name','price_aprox_usd']]
sells_caba_price.head(10)

In [ ]:
#Agrupo las columnas filtradas por lugar y promedio el precio de las propiedades.
sells_caba_group = sells_caba_price.groupby('place_name')
sells_caba_group.mean()

### Cantidad de ventas de propiedades por barrios de Capital Federal.

In [ ]:
#Defino cantidad de ventas por barrio y ordeno de mayor a menor.
sells_caba_group_count = sells_caba_group.count()['state_name']
sells_caba_group_count.sort_values(ascending=False)

Con el conteo de ventas por barrios, se puede observar que Palermo,Belgrano y Caballito son las zonas con más ventas de Capital Federal.
Por otro lado, Las zonas como Villa Soldati, Villa Riachuelo y Villa Real son los barrios con menos ventas.

In [ ]:
#Para tener consistencias en los datos, se decide filtrar los contenidos por un mínimo de ventas por barrio para empezar 
#a analizar.
#como hay min 2 cantidad de ventas y max 1872, filtro por >= 15
sells_caba_group = sells_caba_price.groupby('place_name').agg([np.mean,np.size])
sells_caba_group = sells_caba_group.loc[sells_caba_group[('price_aprox_usd','size')]>=15,:]   
sells_caba_group.head()

### Precio en dólares promedio de una propiedad por barrio en Capital Federal.

#### Precios promedio más altos de Capital Federal.


In [ ]:
#top 10 CABA promedio de precios en Capital Federal.


sells_caba_sort_desc = sells_caba_group.sort_values(('price_aprox_usd','mean'), ascending=False)
sells_caba_sort_desc.head(10)


Claramente, las zonas de Palermo Chico y Puerto Madero son las zonas donde el precio en promedio, dominan en Capital Federal como las zonas promedios más caras de Capital Federal.

In [ ]:
#Visualizacion de precios promedios de los 10 barrios con mayor valor.

plt.ylabel = "1"
ax = sells_caba_sort_desc[('price_aprox_usd','mean')][0:10].plot(title = "Top 10 de precio promedio de ventas en Capital Federal en USD",kind='bar',figsize=(14,4),color= "violet")
ax.set_ylabel('Precio en USD')
ax.set_xlabel('Barrios de Capital Federal')
plt.show()

Con el plot, se ve la gran diferencia de precios promedio entre los barrios de Palermo Chico y Puerto Madero con respecto a los demás barrios que integran el top10 de las zonas más caras.

####  Precios promedio más bajos de Capital Federal.

In [ ]:
#top 10 CABA promedio de precios más bajos en Capital Federal por barrio.

sells_caba_sort_asc = sells_caba_group.sort_values(('price_aprox_usd','mean'), ascending=True)
sells_caba_sort_asc.head(10)

Se puede observar que las zonas de Villa Soldati, Constitución y Villa Riachuelo tienen los precios más bajos
en promedio con los demás barrios.

In [ ]:
#Visualizacion de precios promedios de los 10 barrios con menor valor.
#Villa Soldati es el barrio con menor precio promedio con respecto a los demás barrios de Capital Federal.

plt.ylabel = "1"
ax = sells_caba_sort_asc[('price_aprox_usd','mean')][0:10].plot(title = "Top 10 de las zonas con menor precio promedio de Capital Federal en USD",color = 'violet',kind='bar',figsize=(14,4))
ax.set_ylabel('Precio en USD')
ax.set_xlabel('Barrios de Capital Federal')
plt.show()

Así como Puerto Madero y Palermo Chico mostraban una diferencia de precios muy marcada con los inmediatos seguidores,
en este plot, se puede observar que Villa Soldati se encuentra como la zona de menor precio promedio de Capital Federal, seguidas de Constitución y Villa Riachuelo.

In [ ]:
# Histograma de cantidad de promedio de precios y cantidad de ventas
ax = sells_caba_group[('price_aprox_usd','mean')].hist(figsize=(25,10),color = 'violet');
ax.set_ylabel('Cantidad de propiedades',fontsize=20)
ax.set_xlabel('Rango de precios',fontsize=20)
ax.set_title("Distribución promedio de ventas en Capital Federal",fontsize = 25)
plt.show()

In [ ]:
######################VERRRRR###########################3

#concentracion de precios y cantidad de ventas
sells_caba_group.plot.box(figsize=(15,5));

### Primeros trabajos sobre propiedades en Capital Federal.

In [ ]:
sells_gba_price = sells_gba.loc[:,['state_name','place_name','price_aprox_usd']]
sells_gba_price.head(10)

In [ ]:
sells_gba_group = sells_gba_price.groupby("place_name")
sells_gba_group.mean()

#### Cantidad de ventas de propiedades por barrio en el Gran Buenos Aires.

In [ ]:
#defino cantidad de ventas por barrio
sells_gba_group_count = sells_gba_group.count()['state_name']
sells_gba_group_count.sort_values(ascending=False)

Como primera observación realizada, Nordelta, Tigre, La Plata y Pilar dominan ampliamante el sector de ventas de las zonas de Gran Capital, y su vez, el dataframe nos informa de muchas ventas unitarias de diferentes barrios.

In [ ]:
#como hay min 2 cantidad de ventas y max 2238, filtro por >= 15
sells_gba_group = sells_gba_price.groupby('place_name').agg([np.mean,np.size])
sells_gba_group = sells_gba_group.loc[sells_gba_group[('price_aprox_usd','size')]>=15,:]   
sells_gba_group.head(100)

In [ ]:
#top 10 de los precios promedios más caros por Gran Bueno Aires
sells_gba_sort_desc = sells_gba_group.sort_values(('price_aprox_usd','mean'), ascending=False)
sells_gba_sort_desc.head(10)

In [ ]:
#top 10 GBA promedio
%matplotlib inline

plt.figure()
plt.ylabel = "1"
ax = sells_gba_sort_desc[('price_aprox_usd','mean')][0:10].plot(title = "Top 10 de los precios promedios más caros en localidades del Gran Bueno Aires",color= 'violet',kind='bar',figsize=(14,4))
ax.set_ylabel('Precio en USD')
ax.set_xlabel('Localidades del Gran Buenos Aires')
plt.show()

In [ ]:
#top 10 GBA promedio Ascendente
sells_gba_sort_asc = sells_gba_group.sort_values(('price_aprox_usd','mean'), ascending=True)
sells_gba_sort_asc.head(10)

In [ ]:
#top 10 GBA promedio Ascendente
plt.figure()
plt.ylabel = "1"
ax = sells_gba_sort_asc[('price_aprox_usd','mean')][0:10].plot(title = "Top 10 de los precios promedios más baratos en localidades en el Gran Bueno Aires",color = 'violet',kind='bar',figsize=(14,4))
ax.set_ylabel('Precio en USD')
ax.set_xlabel('Localidades del Gran Buenos Aires')

plt.show()

In [ ]:
# Histograma de cantidad de promedio de precios y cantidad de ventas
#sells_gba_group['price_aprox_usd'].hist(figsize=(16,8));
ax = sells_gba_group[('price_aprox_usd','mean')].hist(figsize=(25,10),color = 'violet');
ax.set_ylabel('Cantidad de propiedades',fontsize=20)
ax.set_xlabel('Rango de precios',fontsize=20)
ax.set_title("Distribución promedio de ventas en GBA",fontsize = 25)
plt.show()

In [ ]:
#concentracion de promedio precios y cantidad de ventas
sells_gba_group.plot.box(figsize=(15,5));

In [ ]:
#cantidad ventas de tipo de propiedades en CABA
sells_caba_property = sells_caba.loc[:,['property_type','place_name']]
sells_caba_property.head(10)

In [ ]:
sells_caba_property = sells_caba.loc[:,['property_type','price_aprox_usd']]
sells_caba_property = sells_caba_property[np.isfinite(sells_caba_property['price_aprox_usd'])]
sells_caba_property['sub_index'] = sells_caba_property.groupby('property_type').cumcount() + 1
sells_caba_property = pd.pivot_table(sells_caba_property,index='sub_index',columns='property_type',values='price_aprox_usd')

sells_caba_property.plot.box(figsize=(15,5))

In [ ]:
#cantidad de ventas por tipo de propiedades en CABA
ax = sells_caba_property_group_count[('place_name','size')][0:10].plot(title = "Distribución de ventas por tipo en Capital Federal",color = 'violet',kind='bar',figsize=(14,4))
ax.set_ylabel('Numero de ventas')
ax.set_xlabel('Tipos de propiedades')

plt.show()

In [ ]:
#cantidad ventas de tipo de propiedades en GBA
sells_gba_property = sells_gba.loc[:,['property_type','price_aprox_usd']]
sells_gba_property = sells_gba_property[np.isfinite(sells_gba_property['price_aprox_usd'])]
sells_gba_property['sub_index'] = sells_gba_property.groupby('property_type').cumcount() + 1
sells_gba_property = pd.pivot_table(sells_gba_property,index='sub_index',columns='property_type',values='price_aprox_usd')

sells_gba_property.plot.box(figsize=(15,5))

In [ ]:
#cantidad de ventas por tipo de propiedades en GBA
ax = sells_gba_property_group_count[('place_name','size')][0:10].plot(title = "Distribución de ventas por tipo en el Gran Buenos Aires",color = 'violet',kind='bar',figsize=(14,4))
ax.set_ylabel('Número de ventas',fontsize=15)
ax.set_xlabel('Tipos de propiedades',fontsize=15)

plt.show()

In [ ]:
########################################################################################

In [ ]:
#Cantidad promedio de habitaciones por propiedad vendida de CABA ordenada de mayor a menor.
sells_caba_rooms_place_name = sells_caba.loc[:,['place_name','rooms']]
sells_caba_rooms_place_name = sells_caba_rooms_place_name[np.isfinite(sells_caba_rooms_place_name['rooms'])]
sells_caba_rooms = sells_caba_rooms_place_name.groupby('place_name').agg([np.mean,np.size])
sells_caba_rooms = sells_caba_rooms.loc[sells_caba_rooms[('rooms','size')]>=5,:].rename(columns={'mean': 'promedio_de_habitacion'})
sells_caba_rooms.sort_values(('rooms','promedio_de_habitacion'), ascending=False).round({('rooms','promedio_de_habitacion'):2})

In [ ]:
#Cantidad promedio de habitaciones por propiedad vendida de GBA ordenada de mayor a menor.
sells_gba_rooms_place_name = sells_gba.loc[:,['place_name','rooms']]
sells_gba_rooms_place_name = sells_gba_rooms_place_name[np.isfinite(sells_gba_rooms_place_name['rooms'])]
sells_gba_rooms = sells_gba_rooms_place_name.groupby('place_name').agg([np.mean,np.size])
sells_gba_rooms = sells_gba_rooms.loc[sells_gba_rooms[('rooms','size')]>=15,:].rename(columns={'mean': 'promedio_de_habitacion'})
#El valor 15, corresponde a que tomo a partir de 15 ventas de propiedades por barrio para tener en cuenta en el promedio.
sells_gba_rooms.sort_values(('rooms','promedio_de_habitacion'), ascending=False).round({('rooms','promedio_de_habitacion'):2})

In [ ]:
#6 - Evolucion global de los precios correlacionada con el dolar? --> Braian
#esta pregunta no la entendí, el que la hizo la explique un poco más y la hago.

In [ ]:
#Cantidad de propiedades dolarizadas por barrio.
sells_caba_por_barrio = sells_caba.loc[:,['place_name','currency']]
sells_caba_por_barrio_con_precio_dolar = sells_caba_por_barrio[sells_caba_por_barrio['currency'] == 'USD'].groupby('place_name').count()
sells_caba_por_barrio_con_precio_dolar = sells_caba_por_barrio_con_precio_dolar.rename(columns={'currency': 'numero_propiedades_con_moneda_dolar'})
sells_caba_por_barrio_con_precio_dolar.sort_values(('numero_propiedades_con_moneda_dolar'),ascending=False).head()

In [ ]:
#Cantidad de propiedades pesificadas por barrio.
sells_caba_por_barrio_con_precio_pesos = sells_caba_por_barrio[sells_caba_por_barrio['currency'] == 'ARS'].groupby('place_name').count()
sells_caba_por_barrio_con_precio_pesos = sells_caba_por_barrio_con_precio_pesos.rename(columns={'currency': 'numero_propiedades_con_moneda_pesos'})
sells_caba_por_barrio_con_precio_pesos.sort_values(('numero_propiedades_con_moneda_pesos'),ascending=False).head()

In [ ]:
#Cantidas de propiedades dolarizadas según su tipo.
sells_caba_por_barrio = sells_caba.loc[:,['property_type','currency']]
sells_caba_por_barrio_con_precio_dolar = sells_caba_por_barrio[sells_caba_por_barrio['currency'] == 'USD'].groupby('property_type').count()
sells_caba_por_barrio_con_precio_dolar = sells_caba_por_barrio_con_precio_dolar.rename(columns={'currency': 'propiedad_en_moneda_dolar'})
sells_caba_por_barrio_con_precio_dolar.sort_values(('propiedad_en_moneda_dolar'),ascending=False).head()

In [ ]:
#Cantidad de propiedades pesificadas según su tipo.
sells_caba_por_barrio = sells_caba.loc[:,['property_type','currency']]
sells_caba_por_barrio_con_precio_pesos = sells_caba_por_barrio[sells_caba_por_barrio['currency'] == 'ARS'].groupby('property_type').count()
sells_caba_por_barrio_con_precio_pesos = sells_caba_por_barrio_con_precio_pesos.rename(columns={'currency': 'propiedad_en_moneda_pesos'})
sells_caba_por_barrio_con_precio_pesos.sort_values(('propiedad_en_moneda_pesos'),ascending=False).head()

In [ ]:
#Distribucion de m2 por barrio (top10)
sells_caba_agrupada_por_sup_y_barrio = sells_caba.loc[:,['place_name','surface_total_in_m2']]
sells_caba_agrupada_por_sup_y_barrio.groupby('place_name').sum().sort_values(('surface_total_in_m2'),ascending=False).head(10)

In [ ]:
#Distribucion promedio de m2 por barrio (top10)
sells_caba_agrupada_por_sup_y_barrio = sells_caba.loc[:,['place_name','surface_total_in_m2']]
sells_caba_agrupada_por_sup_y_barrio.groupby('place_name').mean().sort_values(('surface_total_in_m2'),ascending=False).round({'surface_total_in_m2':2}).head(10)

In [ ]:
#Las expensas promedio mas caras por barrio en Capital Federal.
sells_caba_por_expensas_lugar = sells_caba.loc[:,['place_name','expenses']]
sells_caba_por_expensas_lugar = sells_caba_por_expensas_lugar[(sells_caba_por_expensas_lugar['expenses'] < 100000) & (sells_caba_por_expensas_lugar['expenses'] > 10)]
sells_caba_por_expensas_lugar = sells_caba_por_expensas_lugar[np.isfinite(sells_caba_por_expensas_lugar['expenses'])]
sells_caba_por_expensas_lugar.groupby('place_name').mean().sort_values(('expenses'),ascending=False).round({'expenses':2}).head()

In [ ]:
#Las expensas promedio menos caras por barrio en Capital Federal.
sells_caba_por_expensas_lugar.groupby('place_name').mean().sort_values(('expenses'),ascending=False).round({'expenses':2}).tail()

In [ ]:
#Las expensas promedio mas caras por ciudad del Gran Buenos Aires.
sells_gba_por_expensas_lugar = sells_gba.loc[:,['place_name','expenses']]
sells_gba_por_expensas_lugar = sells_gba_por_expensas_lugar[(sells_gba_por_expensas_lugar['expenses'] < 100000) & (sells_gba_por_expensas_lugar['expenses'] > 10)]
sells_gba_por_expensas_lugar = sells_gba_por_expensas_lugar[np.isfinite(sells_gba_por_expensas_lugar['expenses'])]
sells_gba_por_expensas_lugar = sells_gba_por_expensas_lugar.groupby('place_name').agg([np.mean,np.size])
sells_gba_por_expensas_lugar = sells_gba_por_expensas_lugar.loc[sells_gba_por_expensas_lugar[('expenses','size')]>=15,:]
sells_gba_por_expensas_lugar.sort_values(('expenses','mean'), ascending=False).head(10)

In [ ]:
#Distribucion de stores en Capital Federal
sells_caba_only_store = sells_caba[sells_caba['property_type'] == 'store']
sells_caba_only_store['place_name'].value_counts()

In [ ]:
#Distribucion de stores en Gran Buenos Aires
sells_gba_only_store = sells_gba[sells_gba['property_type'] == 'store']
sells_gba_only_store['place_name'].value_counts()

In [ ]:
#Precio promedio del m2 por barrio en GBA ordenado de mayor a menor
sells_gba_por_ciudades_y_m2 = sells_gba.loc[:,['place_name','price_usd_per_m2']]
sells_gba_por_ciudades_y_m2 = sells_gba_por_ciudades_y_m2[sells_gba_por_ciudades_y_m2['price_usd_per_m2'] < 7000]
sells_gba_por_ciudades_y_m2 = sells_gba_por_ciudades_y_m2[np.isfinite(sells_gba_por_ciudades_y_m2['price_usd_per_m2'])]
sells_gba_por_ciudades_y_m2 = sells_gba_por_ciudades_y_m2.groupby('place_name').agg([np.mean,np.size])
sells_gba_por_ciudades_y_m2 = sells_gba_por_ciudades_y_m2.loc[sells_gba_por_ciudades_y_m2[('price_usd_per_m2','size')]>=15,:]
sells_gba_por_ciudades_y_m2 = sells_gba_por_ciudades_y_m2.rename(columns = {'mean': 'precio_del_m2_por_barrio'})
sells_gba_por_ciudades_y_m2 = sells_gba_por_ciudades_y_m2.sort_values(('price_usd_per_m2','precio_del_m2_por_barrio'), ascending=False)
sells_gba_por_ciudades_y_m2.head(10)

In [ ]:
%matplotlib inline

ax = sells_gba_por_ciudades_y_m2['price_usd_per_m2','precio_del_m2_por_barrio'][:10].plot(title = "Precio promedio del m2 de las zonas mas caras de GBA",color = 'violet',kind='bar',figsize=(14,4))
ax.set_ylabel('Precio USD del metro2')
ax.set_xlabel('Localidades del GBA')
plt.show()

In [ ]:
#Precio promedio del m2 por barrio en CABA ordenado de mayor a menor
sells_caba_por_ciudades_y_m2 = sells_caba.loc[:,['place_name','price_usd_per_m2']]
sells_caba_por_ciudades_y_m2 = sells_caba_por_ciudades_y_m2[sells_caba_por_ciudades_y_m2['price_usd_per_m2'] < 7000]
sells_caba_por_ciudades_y_m2 = sells_caba_por_ciudades_y_m2[np.isfinite(sells_caba_por_ciudades_y_m2['price_usd_per_m2'])]
sells_caba_por_ciudades_y_m2 = sells_caba_por_ciudades_y_m2.groupby('place_name').agg([np.mean,np.size])
sells_caba_por_ciudades_y_m2 = sells_caba_por_ciudades_y_m2.loc[sells_caba_por_ciudades_y_m2[('price_usd_per_m2','size')]>=10,:]
sells_caba_por_ciudades_y_m2 = sells_caba_por_ciudades_y_m2.rename(columns = {'mean': 'precio_del_m2_por_barrio'})
sells_caba_por_ciudades_y_m2 = sells_caba_por_ciudades_y_m2.sort_values(('price_usd_per_m2','precio_del_m2_por_barrio'), ascending=False)
sells_caba_por_ciudades_y_m2.head(10)

In [ ]:
%matplotlib inline

ax = sells_caba_por_ciudades_y_m2['price_usd_per_m2','precio_del_m2_por_barrio'][:10].plot(title = "Precio promedio del m2 de las zonas mas caras de Capital Federal",kind='bar',color='violet',figsize=(14,4))
ax.set_ylabel('Precio USD del metro2')

ax.set_xlabel('Barrios de Capital Federal')
plt.show()

In [ ]:
sells.columns.values

In [ ]:
#CON ESTO SACO EL VALOR DEL DOLAR, CON CADA DATA SET DE 6 MESES, puedo obtener un dolar que irá creciendo y se puede
#almacenar en algun plot del estilo area, en donde se vea la curva de crecimiento en la que seguramente se vera .
sells_caba_with_price_currency_ars = sells_caba[sells_caba['currency'] == 'ARS']
sells_caba_with_price_currency_ars = sells_caba_with_price_currency_ars.loc[:,['price','price_aprox_usd']]
sells_caba_with_price_currency_ars['price'].sum()/sells_caba_with_price_currency_ars['price_aprox_usd'].sum()


### Valor del dólar durante el transcurso del mes Enero del 2017

###### Se inicia un análisis de las propiedades que se encuentran pesificadas, ya que cuentan con su valor tanto en pesos y en dolar.

In [ ]:
#Me quedo con los valores que esten pesificados, ya que con ellos puedo trabajar para obtener el valor del dolar.

only_sells_with_price_arg = sells_caba[sells_caba['currency'] == 'ARS']
only_sells_with_price_arg = only_sells_with_price_arg [(only_sells_with_price_arg['created_on']>'2017-01-01') & (only_sells_with_price_arg['created_on']<'2017-01-31')]
#only_sells_with_price_arg = only_sells_with_price_arg [(only_sells_with_price_arg['created_on']>='2017-01-01') & (only_sells_with_price_arg['created_on']<='2017-01-31')]
#(arriba) esto con la restricciones de filtrado, lo debo quitar.
sells_caba_with_date = only_sells_with_price_arg.loc[:,['created_on','price','price_aprox_usd']]

#Con un valor de una propiedad por día puedo sacar el valor del dolar del mismo día.
sells_caba_with_date = sells_caba_with_date.drop_duplicates(['created_on'], keep='last')
sells_caba_with_date['created_on'] = sells_caba_with_date['created_on'].astype("datetime64")
sells_caba_with_date = sells_caba_with_date.sort_values(('created_on'), ascending=True)

#Agrego una columna donde voy a calcular el dolar del día.
sells_caba_with_date['dollar'] = sells_caba_with_date['price']/sells_caba_with_date['price_aprox_usd']
sells_caba_with_date = sells_caba_with_date.round({'dollar':2})
sells_caba_with_date.set_index('created_on', inplace=True)

In [ ]:

bar = sells_caba_with_date['dollar'].plot.bar(figsize=(8,8), fontsize=15,color = 'orange',stacked=True)
bar.set_xticklabels(sells_caba_with_date.index.format()) # formato YYYY-MM-DD
bar.set_xlabel('Fecha(YYYY-MM-DD)', fontsize=15)
bar.set_ylabel('USD',fontsize = 15)
 
plt.show()

Con este plot se puede ver claramente que durante el mes de Enero del 2017, el precio del dolar se mantuvo estable, por lo que se puede decir que en el mercado de las propiedades, el valor de las mismas no vario mucho con respecto a las anteriores.